# 📊 AO3 Wrapped: Your Reading Year in Review

Explore and visualize your personal reading history from Archive of Our Own — see your fandom trends, favorite works, and reading patterns throughout the year.



---



## ⚠️ **Important:** Make a Copy First

Before running this notebook, it’s recommended to **make your own copy**:

1. Click **File → Save a copy in Drive**.
2. Open the copy in your Google Drive.

This ensures:

- Your AO3 credentials and data stay private.
- You can freely **customize settings** if you want.



---



## 📑 Table of Contents

1. **Setup** - Install dependencies and configure the environment.
2. **Security & Privacy** - Important information about your data and credentials.
3. **Login** - Authenticate with your AO3 account.
4. **Data Collection** - Scrape your reading history or load existing data from CSV files.
5. **Visualizations** — Your reading patterns at a glance:
   - Summary stats
   - Daily activity heatmap
   - Deleted works over time
   - Fandom trends
   - Most visited works & authors
   - Tag & fandom word clouds



---


## ⚙️ Quick Start

1. Run the **Setup** cell to install all requirements.
2. Run the **Login** cell to authenticate with AO3.
3. Choose to either **scrape new data** or **load existing data files**.
4. Run the visualization cells to see your AO3 Wrapped!

---

# 1. Setup
Click the **Play button** (▶️) on the code cell below to install all required packages and configure the environment. **You must run this cell before proceeding.**

* This cell can take **1-2 minutes** to complete, especially on the first run.
* It's installing necessary components like Python packages and special fonts for different languages.
* Please wait for the **"✅ Setup complete!"** message before moving on.



In [ ]:
# @title SETUP & CONFIG: Install, Import, and Configure Environment (RUN ONCE)
"""
🔧 SETUP INSTRUCTIONS:
1. Run this cell once (may take 30–60 seconds, or longer depending on network)
2. Wait for the "✅ Setup complete!" message
3. Proceed to login and analysis cells

📋 This cell:
- Installs required Python packages
- Installs Noto Sans CJK fonts (for CJK text support)
- Imports all libraries
- Configures matplotlib fonts with smart fallback
- Loads analysis/visualization parameters
"""

print("Installing packages...")

# --- Package installation ---
import subprocess
import sys

packages = {
    "beautifulsoup4": "bs4",
    "requests": "requests",
    "tqdm": "tqdm",
    "seaborn": "seaborn",
    "wordcloud": "wordcloud",
    "Pillow": "PIL",
}

for pip_name, import_name in packages.items():
    try:
        __import__(import_name)
        print(f"  ✓ {pip_name} already installed")
    except ImportError:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-q", pip_name]
        )
        print(f"  ✓ {pip_name} installed")

# --- Install CJK fonts ---
print("Installing CJK fonts...")
subprocess.run(['apt-get', 'update', '-qq'],
               stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run(['apt-get', 'install', '-y', 'fonts-noto-cjk'],
               stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print("  ✓ Fonts installed")

# --- Import libraries ---
print("Importing libraries...")

# Standard library
import os
import re
import ast
import time
import getpass
from collections import Counter

from google.colab import drive, files

# Data manipulation
import pandas as pd
import numpy as np

# Web scraping
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

# Visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from matplotlib.ticker import FixedLocator
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
from matplotlib import colormaps

# Word cloud
from wordcloud import WordCloud
from PIL import Image

print("  ✓ Libraries imported")

# ========================================
# ⚙️ CONFIGURATION SETTINGS (Analysis & Visualization)
# ========================================

# --- Scraping Settings ---
YEAR_LIMIT = 2025
SCRAPE_DELAY = 3  # seconds between requests
MAX_PAGES=None

# --- Visualization Settings ---
NUM_TOP_FANDOMS = 8
MAX_WORDCLOUD_WORDS = 100
TITLE_FONT_SIZE = 22
SUBTITLE_FONT_SIZE = 16
PADDING = 50
OUTPUT_DPI = 300

# --- AO3 Color Palette ---
AO3_BG_LIGHT = '#FFFFFF'
AO3_TEXT_DARK = '#000000'
AO3_ACCENT_RED = '#990000'
AO3_GRID_LIGHT = '#EEEEEE'
AO3_OTHER_GRAY = '#C0C0C0'
AO3_GA = "#94c90e"
AO3_TEEN = "#ead70f"
AO3_MATURE = "#eb7400"
AO3_EXPLICIT = '#a80403'

# Custom colormap for heatmaps
colors = [AO3_GRID_LIGHT, "#FFC8C8", "#CC0000", AO3_ACCENT_RED]
AO3_HEATMAP_CMAP = LinearSegmentedColormap.from_list("AO3_Heatmap", colors, N=256)

# --- Global matplotlib settings (before font lock) ---
plt.rcParams.update({
    'font.size': 14,
    'text.color': AO3_TEXT_DARK,
    'axes.labelcolor': AO3_TEXT_DARK,
    'xtick.color': AO3_TEXT_DARK,
    'ytick.color': AO3_TEXT_DARK,
    'figure.facecolor': AO3_BG_LIGHT,
    'axes.facecolor': AO3_BG_LIGHT,
    'axes.unicode_minus': False
})

# ------------------------------------------------------------------
# 🔤 FONT CONFIGURATION (Merged, Cached, Locked for Exports)
# ------------------------------------------------------------------

print("Configuring fonts...")

_FONT_CACHE = None

def setup_and_find_best_font():
    """
    Install, register, and select the best available font.

    Priority order:
    1. Bold Noto Sans CJK
    2. Regular Noto Sans CJK
    3. Other Unicode-safe fonts (Noto Sans, Arial Unicode, DejaVu Sans)

    Returns:
        font_path (str | None): Path to selected font file
        font_name (str): Font family name for matplotlib
    """
    global _FONT_CACHE

    if _FONT_CACHE is not None:
        return _FONT_CACHE

    # Register known Noto Sans CJK locations
    possible_paths = [
        "/usr/share/fonts/opentype/noto/NotoSansCJK-Bold.ttc",
        "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc",
        "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.otf",
        "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttf",
    ]

    for p in possible_paths:
        if os.path.exists(p):
            fm.fontManager.addfont(p)

    # Reload font manager to ensure discovery
    fm._load_fontmanager(try_read_cache=False)

    # Discover system fonts
    fonts = (
        fm.findSystemFonts(fontext="ttf") +
        fm.findSystemFonts(fontext="ttc")
    )

    priority = [
        "notosanscjk",
        "notosans",
        "arialunicode",
        "dejavusans",
    ]

    fonts_lower = [f.lower() for f in fonts]

    # Prefer bold
    for key in priority:
        for i, name in enumerate(fonts_lower):
            if key in name and "bold" in name:
                _FONT_CACHE = (fonts[i], fm.FontProperties(fname=fonts[i]).get_name())
                break
        if _FONT_CACHE is not None:
            break

    # Fallback to regular
    if _FONT_CACHE is None:
        for key in priority:
            for i, name in enumerate(fonts_lower):
                if key in name:
                    _FONT_CACHE = (fonts[i], fm.FontProperties(fname=fonts[i]).get_name())
                    break
            if _FONT_CACHE is not None:
                break

    # Absolute fallback
    if _FONT_CACHE is None:
        if fonts:
            _FONT_CACHE = (fonts[0], fm.FontProperties(fname=fonts[0]).get_name())
        else:
            _FONT_CACHE = (None, "sans-serif")

    # Apply global matplotlib settings to lock typography
    font_path, font_name = _FONT_CACHE
    if font_path is not None:
        prop = fm.FontProperties(fname=font_path)
        plt.rcParams['font.family'] = prop.get_name()
        plt.rcParams['font.sans-serif'] = [prop.get_name()]
    else:
        plt.rcParams['font.family'] = font_name
        plt.rcParams['font.sans-serif'] = [font_name]

    # Ensure PDFs and SVGs embed fonts for reproducibility
    plt.rcParams['pdf.fonttype'] = 42  # TrueType
    plt.rcParams['svg.fonttype'] = 'none'  # embed font paths

    return _FONT_CACHE

# Initialize font configuration
FONT_PATH, FONT_NAME = setup_and_find_best_font()
print(f"  ✓ Font configured: {FONT_NAME}")

print(" Configuration loaded")
print("\n ✅ Setup complete! You can now proceed to login and analysis.")

# 2. Security & Privacy FAQ

### Is it safe to enter my AO3 password?

**Yes, when used as intended:**
- This notebook runs **entirely in your private Google Colab session**
- Your password is **never stored or logged**
- Your credentials are only used to authenticate with AO3, then **cleared immediately**

### Can the notebook creator see my password?

**No:**
- Each Colab session is **isolated to your account**
- The creator has **no access** to your inputs, data, or runtime

### What happens to my password after login?
- Used once for authentication
- **Deleted from memory right after**
- Not saved to files, variables, or outputs

### Still concerned?

**You can:**
1. Review the login code `ao3_login()`
2. Use a temporary AO3 password
3. Download and run the notebook locally

### Security Best Practices
- Clear outputs before sharing (`Edit > Clear all outputs`)
- Use a strong, unique AO3 password


# 3. Login

In [ ]:
#@title Log in to AO3 { display-mode: "form" }

def ao3_login(username, password):
    """
    Logs into AO3 using requests.Session and returns the authenticated session.
    Automatically extracts authenticity_token.
    """
    session = requests.Session()
    session.headers.update({"User-Agent": "Mozilla/5.0"})

    login_page = session.get("https://archiveofourown.org/users/login")
    soup = BeautifulSoup(login_page.text, "html.parser")

    token = soup.find("input", {"name": "authenticity_token"})["value"]

    payload = {
        "authenticity_token": token,
        "user[login]": username,
        "user[password]": password,
    }

    # Submit login form
    session.post("https://archiveofourown.org/users/login", data=payload)

    return session

#@markdown ---
#@markdown ### 🔐 AO3 Authentication
#@markdown
#@markdown **🔒 Your Privacy Matters:**
#@markdown - Your password is entered securely and **never visible** in the notebook
#@markdown - Credentials are **only used in this session** and are **never stored**
#@markdown - This notebook runs entirely in **your private Google Colab environment**
#@markdown - Your data never leaves your session
#@markdown
#@markdown **📝 Instructions:**
#@markdown 1. Click the ▶️ **Play button** on the left side of this cell
#@markdown 2. A box for `AO3 Username:` will appear.
#@markdown    - **Type your username, then press the `Enter` key.**
#@markdown 3. A new box for `AO3 Password:` will appear.
#@markdown    - **Type your password, then press the `Enter` key.** (Note: your password will be masked for security).
#@markdown 4. Wait for the "✅ Login Successful!" message. (may take about 1min depending on network speed)
#@markdown
#@markdown ---

print("🔐 AO3 Login\n")
print("Enter your credentials below (password will be hidden):\n")

try:
    username = input("AO3 Username: ").strip()
    password = getpass.getpass("AO3 Password (hidden): ")

    if not username or not password:
        print("\n❌ Both username and password are required.")
    else:
        print(f"\n🔄 Logging in as '{username}'...\n")

        # Attempt login
        my_session = ao3_login(username, password)

        # Verify login was successful
        test_page = my_session.get(f"https://archiveofourown.org/users/{username}/readings")

        if 'Log In' in test_page.text:
            print("❌ Login Failed\n")
            print("Possible reasons:")
            print("  • Incorrect username or password")
            print("  • AO3 might be down or slow")
            print("  • Check if you can log in at archiveofourown.org")
            print("\n💡 Tip: Copy-paste your credentials to avoid typos")
        else:
            print("✅ Login Successful!\n")
            print(f"   Logged in as: {username}")
            print("   You can now proceed to download your history.")

            # Clear password from memory
            del password

except KeyboardInterrupt:
    print("\n⚠️ Login cancelled by user.")
except Exception as e:
    print(f"\n❌ Unexpected error: {e}")
    print("   Please try again or check your internet connection.")
finally:
    # Ensure password is cleared even if error occurs
    try:
        del password
    except:
        pass

# 4. Data Collection

In [ ]:
# @title Run Scraping { display-mode: "form" }
#@markdown ---
#@markdown ### 🔄 Scrape Your Reading History
#@markdown
#@markdown **What this does:**
#@markdown - Fetches your AO3 reading history page by page
#@markdown - Extracts work details, tags, and statistics
#@markdown - Tracks deleted works you've encountered
#@markdown
#@markdown **Progress:**
#@markdown - A progress bar will show scraping status
#@markdown - This may take 15-20 minutes for ~100 pages
#@markdown - Please be patient and don't interrupt!
#@markdown
#@markdown ---

import time
import re
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

def scrape_readings(session, username, year_limit=2025, delay=3, start_page=1, max_pages=None):
    """
    Scrape AO3 reading history using an existing session object.

    :param session: A requests.Session object that is already logged into AO3.
    :param username: The AO3 username whose history is being scraped.
    :param year_limit: Stop scraping when a work older than this year is encountered.
    :param delay: Time (in seconds) to wait between page requests.
    :param start_page: The page number to begin scraping from.
    :param max_pages: Maximum number of pages to scrape, regardless of last_page.
    :return: Tuple of (results, deleted_results)
    """

    base_url = f"https://archiveofourown.org/users/{username}/readings?page="

    results = []
    deleted_results = []
    page = start_page
    last_page = None
    year_limit_reached = False
    printed_first_work = False

    # Initial page fetch to determine total pages
    print(" Determining total pages...")
    initial_url = base_url + str(start_page)
    r = session.get(initial_url)

    if r.status_code != 200:
        raise Exception(f"Failed to fetch initial page: HTTP {r.status_code}")

    soup = BeautifulSoup(r.text, "html.parser")

    # Check for login/error page
    page_title = soup.find('title').text.strip() if soup.find('title') else ''
    if 'Log In' in page_title or 'Account Access' in page_title:
        raise Exception("Session is invalid. Please log in again.")

    # Determine last page
    nav = soup.find("ol", role="navigation")
    if nav:
        links = nav.find_all("a")
        if links and links[-1].text.strip() == 'Next →':
            last_page = int(links[-2].text) if len(links) > 2 else 1
        elif links:
            try:
                last_page = int(links[-1].text)
            except ValueError:
                last_page = 1
        else:
            last_page = 1
    else:
        last_page = 1

    if max_pages is not None:
        last_page = min(last_page, start_page + max_pages - 1)

    print(f" Reading History Found")
    print(f"   If Year limit is not reached, scrape to page: {last_page}")
    print(f"   Year limit: {year_limit}")

    # Create progress bar
    pbar = tqdm(
        range(start_page, last_page + 1),
        desc="📖 Scraping pages",
        unit="page",
        ncols=100,
        bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} pages [{elapsed}<{remaining}]'
    )

    # Process initial page (already fetched)
    page_soups = {start_page: soup}

    for page in pbar:
        # Use already-fetched soup for first page, otherwise fetch new
        if page not in page_soups:
            url = base_url + str(page)
            r = session.get(url)

            if r.status_code == 429:
                pbar.write(" Rate-limited. Sleeping 60 seconds...")
                time.sleep(60)
                r = session.get(url)  # Retry after waiting

            if r.status_code != 200:
                pbar.write(f" Failed to fetch page {page}: HTTP {r.status_code}")
                continue

            soup = BeautifulSoup(r.text, "html.parser")
        else:
            soup = page_soups[page]

        # Find works on page
        work_list = soup.find("ol", class_="reading work index group") or soup.find("ol", class_="work index group")
        works = work_list.find_all("li", class_=re.compile(r"work blurb group")) if work_list else []

        if not works:
            pbar.write(f" No works found on page {page}")
            break

        # Update progress bar description with current stats
        pbar.set_postfix({
            'works': len(results),
            'deleted': len(deleted_results)
        })

        for i, work in enumerate(works):
            # Detect deleted works
            classes = work.get("class", [])
            if any("deleted" in c for c in classes):
                viewed = work.find("h4", class_="viewed heading") or work.find("h4", class_="viewed")
                raw_text = viewed.get_text(" ", strip=True) if viewed else ""
                m_deleted = re.search(r"last visited\s*([\d]{1,2}\s[A-Za-z]{3}\s\d{4})", raw_text, re.IGNORECASE)
                last_visited_date = m_deleted.group(1) if m_deleted else None

                deleted_results.append({
                    "page": page,
                    "index": i,
                    "last_visited_date": last_visited_date
                })
                continue

            # Work ID
            work_id = None
            work_id_attr = work.get('id')
            if work_id_attr and work_id_attr.startswith('work_'):
                try:
                    work_id = int(work_id_attr.split('_')[-1])
                except ValueError:
                    pass

            # Title & Author
            h4 = work.find("h4", class_="heading")
            if not h4 or not h4.a:
                continue
            title = h4.a.text.strip()
            author_tag = h4.find("a", rel="author")
            author = author_tag.text.strip() if author_tag else "Anonymous"

            # Print the first non-deleted work once (for verification)
            if not printed_first_work:
                pbar.write(f"✓ First work found: '{title}' by {author}")
                printed_first_work = True

            # Fandoms
            fandoms = []
            h5 = work.find("h5", class_="fandoms heading")
            if h5:
                fandoms = [a.text.strip() for a in h5.find_all("a", class_="tag")]

            # Rating & Category (from required-tags)
            required_tags_ul = work.find("ul", class_="required-tags")
            rating = None
            category = None
            if required_tags_ul:
                rating_tag = required_tags_ul.find("span", class_=re.compile(r"rating-(.+)"))
                if rating_tag:
                    rating = rating_tag.text.strip()
                category_tag = required_tags_ul.find("span", class_=re.compile(r"category-(.+)"))
                if category_tag:
                    category = category_tag.text.strip()

                # Work completion status
                complete_tag = required_tags_ul.find("span", class_=re.compile(r"complete-(yes|no)"))
                if complete_tag:
                    complete_classes = complete_tag.get("class", [])
                    if "complete-yes" in complete_classes:
                        is_complete = True
                    elif "complete-no" in complete_classes:
                        is_complete = False

            # Tags (relationships, characters, freeforms, warnings)
            tags_ul = work.find("ul", class_="tags commas")
            relationships = []
            characters = []
            freeforms = []
            warnings = []
            if tags_ul:
                for li in tags_ul.find_all("li"):
                    cls = li.get("class", [])
                    a_tag = li.find("a", class_="tag")
                    if not a_tag:
                        continue
                    tag_text = a_tag.text.strip()
                    if "relationships" in cls:
                        relationships.append(tag_text)
                    elif "characters" in cls:
                        characters.append(tag_text)
                    elif "freeforms" in cls:
                        freeforms.append(tag_text)
                    elif "warnings" in cls:
                        warnings.append(tag_text)

            # Summary text
            # summary_text = None
            # summary_blockquote = work.find("blockquote", class_="userstuff summary")
            # if summary_blockquote:
            #     paragraphs = summary_blockquote.find_all("p")
            #     summary_text = "\n".join(p.get_text(strip=True) for p in paragraphs)
            #     if not summary_text:
            #         summary_text = summary_blockquote.get_text(strip=True)

            # Stats
            stats_dl = work.find("dl", class_="stats")
            def get_stat(class_name):
                dd = stats_dl.find("dd", class_=class_name) if stats_dl else None
                if dd:
                    text = dd.get_text(strip=True).replace(",", "")
                    if text:
                        try:
                            return int(text)
                        except ValueError:
                            return 0
                return 0

            word_count = get_stat("words")
            hits = get_stat("hits")
            comments = get_stat("comments")
            kudos = get_stat("kudos")
            bookmarks = get_stat("bookmarks")
            language_dd = stats_dl.find("dd", class_="language") if stats_dl else None
            language = language_dd.text.strip() if language_dd else "Unknown"

            # Chapter information
            chapters = None
            chapters_dd = stats_dl.find("dd", class_="chapters") if stats_dl else None
            if chapters_dd:
                chapters = chapters_dd.text.strip()

            # Visit info, year, and full date extraction
            viewed_h4 = work.find("h4", class_="viewed heading")
            if not viewed_h4:
                viewed_h4 = work.find("h4", class_="viewed")
            if not viewed_h4:
                continue

            viewed_text = viewed_h4.get_text(" ", strip=True)

            # Full read date
            m_date_full = re.search(r"Last visited[:\s]*([\d]{1,2}\s[A-Za-z]{3}\s[\d]{4})", viewed_text, re.IGNORECASE)
            full_read_date = m_date_full.group(1).strip() if m_date_full else None

            # Year extraction for limit check
            if full_read_date:
                m_year = re.search(r"(\d{4})$", full_read_date)
                year = int(m_year.group(1)) if m_year else None
            else:
                m_year_fallback = re.search(r"\b(20\d{2})\b", viewed_text)
                year = int(m_year_fallback.group(1)) if m_year_fallback else None

            if year is None or year < year_limit:
                pbar.write(f"📅 Reached year limit (found work from {year})")
                year_limit_reached = True
                break

            # Visits count
            m_visits = re.search(r"Visited\s(once|\d+)", viewed_text)
            visited_count = 1 if (m_visits and m_visits.group(1) == "once") else (int(m_visits.group(1)) if m_visits else 0)

            # Extract datetime
            dt_tag = work.find("p", class_="datetime")
            datetime_value = dt_tag.text.strip() if dt_tag else None

            # Build work data
            work_data = {
                "work_id": work_id,
                "title": title,
                "author": author,
                "rating": rating,
                "category": category,
                "fandoms": fandoms,
                "relationships": relationships,
                "characters": characters,
                "freeforms": freeforms,
                "warnings": warnings,
                # "summary": summary_text,
                "is_complete": is_complete,
                "chapters": chapters,
                "word_count": word_count,
                "visited_count": visited_count,
                "language": language,
                "hits": hits,
                "comments": comments,
                "kudos": kudos,
                "bookmarks": bookmarks,
                "year": year,
                "full_read_date": full_read_date,
                "last_updated": datetime_value,
                "page": page
            }

            results.append(work_data)

        # Check if we broke out due to year limit
        if year_limit_reached:
            pbar.write("✓ Year limit reached, stopping scrape")
            break

        # Add delay between pages (except for last page)
        if page < last_page:
            time.sleep(delay)

    pbar.close()
    return results, deleted_results


# Execute scraping
try:
    print(f" Starting scrape for user: {username}")
    print(f" Year limit: {YEAR_LIMIT}")
    print(f"⏱  Delay between requests: {SCRAPE_DELAY}s\n")

    results, deleted_results = scrape_readings(
        session=my_session,
        username=username,
        year_limit=YEAR_LIMIT,
        delay=SCRAPE_DELAY,
        start_page=1,
        max_pages=MAX_PAGES
    )

    print(f"\n Scraping complete!")
    print(f"    Collected {len(results)} works")
    print(f"    Found {len(deleted_results)} deleted works")

    if len(results) == 0:
        print("\n WARNING: No works found. This might indicate:")
        print("   • Your reading history is empty for this year")
        print("   • There was a login or scraping issue")
        print("   • AO3 might be experiencing issues")
    # else:
    #     # Show sample of collected data
    #     print(f"\n Date range: {min(r['full_read_date'] for r in results)} to {max(r['full_read_date'] for r in results)}")

    # Create DataFrames
    df = pd.DataFrame(results)
    df_del = pd.DataFrame(deleted_results)

    print("\n✓ DataFrames created and ready for analysis!")

except KeyboardInterrupt:
    print("\n Scraping interrupted by user")
    print(f"   Partial results: {len(results)} works collected")
    df = pd.DataFrame(results) if results else pd.DataFrame()
    df_del = pd.DataFrame(deleted_results) if deleted_results else pd.DataFrame()

except Exception as e:
    print(f"\n Scraping failed: {e}")
    print("   Please check your internet connection and try again.")
    print("   If the problem persists, try logging in again.")
    raise

In [ ]:
# @title Save to CSV File
df.to_csv('ao3_history_2025.csv', index=False)
df_del.to_csv('ao3_history_2025_deleted.csv', index=False)

from google.colab import files
files.download('ao3_history_2025.csv')
files.download('ao3_history_2025_deleted.csv')

# Load Data (SKIP IN 1ST RUN)

If you have previously scraped your data and saved it, or have existing `ao3_history.csv` and `ao3_history_deleted.csv` files, you can load them here instead of re-scraping.

You have two options for loading your files:
### Option 1: Direct File Upload (Default)
This is the easiest method. When you run the code cell below, two separate file upload dialogs will automatically appear.
1.  **Run the code cell below.**
2.  When the **first upload dialog** appears, select and upload your **ao3_history** file (e.g., `ao3_history_2025.csv`).
3.  When the **second upload dialog** appears, select and upload your **ao3_history_deleted`** file.

### Option 2: Load from Google Drive
Choose this option if your CSV files are already stored in your Google Drive.
1.  **Modify the code cell below:** Change the line `use_drive = False` to `use_drive = True`.
2.  **Run the modified code cell.**
3.  You will be prompted to connect to your Google Drive. Follow the instructions to authenticate.
4.  Once connected, you'll be asked to **enter the full path** to your ao3_history file within your Google Drive (e.g., `/content/drive/MyDrive/my_ao3_data/ao3_history_2025.csv`).
5.  Then, enter the full path to your `ao3_history_deleted.csv` file.

**Important Notes:**
*   Ensure the CSV files you use are the correct ones.
*   The notebook expects two CSV files: one for your main reading history and one for deleted works.

In [ ]:
# @title Load Data (OPTIONAL)
"""
INSTRUCTIONS:
1. Upload your CSV files to Google Drive
2. Mount your drive below
3. Update the file paths

OR use the file upload option if you prefer
"""
# Option 1: Load from Google Drive
use_drive = False  # Set to False to upload files directly

if use_drive:
    from google.colab import drive, files
    drive.mount('/content/drive')

    # UPDATE THESE PATHS to match your file locations
    file_path = input("Enter path to ao3_history CSV: ").strip()
    if not file_path:
        print("please input file path")

    deleted_work_path = input("Enter path to deleted works CSV: ").strip()
    if not deleted_work_path:
        print("please input deleted work path")
else:
    # Option 2: Upload files directly
    print("Please upload your ao3_history CSV file:")
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]

    print("Please upload your deleted works CSV file:")
    uploaded = files.upload()
    deleted_work_path = list(uploaded.keys())[0]

# Load the files
try:
    df = pd.read_csv(file_path)
    df_del = pd.read_csv(deleted_work_path)
    print(f"✅ Loaded {len(df)} works and {len(df_del)} deleted works")
except FileNotFoundError as e:
    print(f"❌ Error: File not found - {e}")
except Exception as e:
    print(f"❌ Error loading data: {e}")

# 5. Data Visualization

Run each code cell in this section individually to generate and display the different graphs and insights. Each visualization will also be saved automatically as a PNG image in your output folder.

In [ ]:
# @title Data Processing (MUST RUN before proceeding)

# ------------------------------------------------------------------
# Helper function for consistent text in matplotlib
# ------------------------------------------------------------------
def add_text_locked(
    obj,            # Figure or Axes
    x, y,           # Position
    text,           # Text string
    fontsize=14,
    fontweight="normal",
    color=AO3_TEXT_DARK,
    ha="left",
    va="center",
    style="normal",
    alpha=1.0,
    is_title=False,  # True if calling for ax.set_title
    pad=0            # Optional padding for titles
):
    """
    Add text to a matplotlib Figure or Axes using the locked font.

    Parameters:
        obj: matplotlib.figure.Figure or matplotlib.axes.Axes
        x, y: Position (fraction for fig, axes coords for ax)
        text: Text string
        fontsize, fontweight, color, ha, va, style, alpha: Text style
        is_title: If True and obj is Axes, calls ax.set_title()
        pad: Title padding
    """
    prop = fm.FontProperties(fname=FONT_PATH)

    if is_title and isinstance(obj, plt.Axes):
        obj.set_title(
            text,
            fontproperties=prop,
            fontsize=fontsize,
            fontweight=fontweight,
            color=color,
            pad=pad,
            loc=ha  # 'left', 'center', 'right'
        )
    elif isinstance(obj, plt.Axes):
        obj.text(
            x, y, text,
            transform=obj.transAxes,
            fontsize=fontsize,
            fontweight=fontweight,
            color=color,
            ha=ha,
            va=va,
            fontproperties=prop,
            style=style,
            alpha=alpha
        )
    elif isinstance(obj, plt.Figure):
        obj.text(
            x, y, text,
            fontsize=fontsize,
            fontweight=fontweight,
            color=color,
            ha=ha,
            va=va,
            fontproperties=prop,
            style=style,
            alpha=alpha
        )
    else:
        raise TypeError("obj must be a Figure or Axes")


# --- 1. Reusable Data Processing Utilities ---

def get_daily_data(df, value_col, agg_func='count'):
    """Aggregates a metric daily. Uses 'full_read_date' index."""

    # Ensure date column is the index for aggregation
    if 'full_read_date' not in df.columns:
        df = df.reset_index(names='full_read_date') # assuming date is in index

    df['full_read_date'] = pd.to_datetime(df['full_read_date'])
    daily_data = df.groupby('full_read_date').agg(
        daily_metric=(value_col, agg_func)
    )['daily_metric']

    return daily_data

def reindex_time_series(daily_series):
    """Fills in missing dates within the range with 0s."""
    idx = daily_series.index
    full_date_range = pd.date_range(start=idx.min(), end=idx.max(), freq='D')

    # Reindex and fill NaN with 0
    reindexed_series = daily_series.reindex(full_date_range, fill_value=0)
    reindexed_series.index.name = 'full_read_date'

    # Add calendar components needed for pivot
    df_reindexed = reindexed_series.to_frame(name='daily_metric')
    df_reindexed['dayofweek'] = df_reindexed.index.dayofweek # 0=Mon, 6=Sun
    df_reindexed['weekofyear'] = df_reindexed.index.isocalendar().week.astype(int)

    return df_reindexed

def get_month_annotations(date_index):
    """Calculates X-positions and labels for month annotations."""
    month_starts = []
    month_labels = []

    first_date = date_index.min()
    last_date = date_index.max()

    # Find the starting week number of the dataset (needed for column index calculation)
    min_week = first_date.isocalendar().week

    # Iterate through every month in the data range
    for month in pd.date_range(start=first_date, end=last_date, freq='MS'):
        start_week = month.isocalendar().week

        # Column index relative to the first week of the heatmap
        col_index = start_week - min_week

        month_starts.append(col_index + 0.5)
        month_labels.append(month.strftime('%b'))

    return month_starts, month_labels

def get_monthly_top_categories(df, category_col, num_top=3, other_label='Other'):
    """
    Identifies the top N categories WITHIN EACH MONTH.
    Returns the actual top performers for each month, with all other categories grouped as 'Other'.
    Excludes empty/null tags.
    """
    df_temp = df.copy()
    # --- Robustness Check: Ensure 'full_read_date' is a column, not the index ---
    if isinstance(df_temp.index, pd.DatetimeIndex) and df_temp.index.name == 'full_read_date':
        df_temp = df_temp.reset_index()

    # 1. Prepare Data and Explode
    # Ensure 'full_read_date' is datetime type (re-casting just in case)
    df_temp['full_read_date'] = pd.to_datetime(df_temp['full_read_date'])

    # Create the 'month' Period column
    df_temp['month'] = df_temp['full_read_date'].dt.to_period('M')

    # Explode the list-like column
    df_exploded = df_temp.explode(category_col)
    # Filter out null, empty string, or list-like empty tags
    df_exploded = df_exploded[df_exploded[category_col].astype(str).str.strip().ne('')]
    df_exploded = df_exploded.dropna(subset=[category_col])

    # 2. Calculate Monthly Counts
    monthly_counts = df_exploded.groupby(['month', category_col]).agg(
        fic_count=('work_id', 'nunique')
    ).reset_index()

    # 3. Determine Top N Categories FOR EACH MONTH
    def get_top_n(group, month_val):
        # Sort by fic_count to identify the top N categories for this month
        sorted_group = group.sort_values(by='fic_count', ascending=False)
        top_n_categories = sorted_group.head(num_top)[category_col].tolist()

        # Get the top N rows
        top_n_group = sorted_group.head(num_top).copy()

        # Calculate the sum of all categories NOT in the top N for this month
        not_in_top = ~group[category_col].isin(top_n_categories)
        other_sum = group.loc[not_in_top, 'fic_count'].sum()

        # Create the 'Other' row if there are other categories
        if other_sum > 0:
            other_row = pd.DataFrame({
                'month': [month_val],
                category_col: [other_label],
                'fic_count': [other_sum]
            })
            # Combine the monthly top N with the monthly 'Other' sum
            top_n_group = pd.concat([top_n_group, other_row], ignore_index=True)

        return top_n_group

    # Apply the ranking/grouping function
    final_monthly_data = monthly_counts.groupby('month', group_keys=False).apply(
        lambda group: get_top_n(group, group.name)
    ).reset_index(drop=True)

    # 4. Get all unique categories that appear in top N across any month
    all_top_categories = final_monthly_data[
        final_monthly_data[category_col] != other_label
    ][category_col].unique().tolist()

    # 5. Pivot for the stacked chart format
    pivot_data = final_monthly_data.pivot(
        index='month',
        columns=category_col,
        values='fic_count'
    ).fillna(0)

    return pivot_data, all_top_categories

# --------- TAGS PROCESSING --------------------------
def safe_parse_list(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return []
    if isinstance(x, list):
        return x
    if isinstance(x, np.ndarray):
        return x.tolist()
    if isinstance(x, str):
        x = x.strip()
        if x in ('', '[]'):
            return []
        try:
            return ast.literal_eval(x)
        except Exception:
            return []
    return []


def clean_fandom(fandom_str):
    """
    Clean a single fandom string to extract the core fandom name.

    Examples:
        "全职高手 - 蝴蝶蓝 | Quánzhí Gāoshǒu - Húdié Lán" -> "全职高手"
        "全职高手 | The King's Avatar (Cartoon)" -> "全职高手"
        "崩坏：星穹铁道 | Honkai: Star Rail (Video Game)" -> "崩坏：星穹铁道"
        "Original Work" -> "Original Work"
        "原创作品" -> "Original Work"
    """
    if not fandom_str:
        return None

    # Standardize Original Work variations first
    if fandom_str in ('原创作品', 'Original Works'):
        return 'Original Work'

    # Split on | to separate bilingual parts
    parts = [p.strip() for p in fandom_str.split('|')]

    # Take the first part (usually the original language)
    core = parts[0]

    # Remove everything after " - " (author/source info)
    # Example: "全职高手 - 蝴蝶蓝" -> "全职高手"
    if ' - ' in core:
        core = core.split(' - ')[0].strip()

    # Remove parenthetical info (Video Game, TV, Cartoon, etc.)
    core = re.sub(r'\s*\([^\)]*\)$', '', core).strip()

    return core if core else None


def clean_tag_generic(tag_str):
    """Standard cleaner for Characters, Relationships, and Freeforms."""
    if not tag_str or pd.isna(tag_str): return None
    # Remove AO3 suffixes and parentheticals
    tag_str = re.sub(r'\s*-\s*(Character|角色|Relationship|Freeform)$', '',
                     str(tag_str), flags=re.IGNORECASE)
    tag_str = re.sub(r'\s*\([^\)]*\)$', '', tag_str).strip()
    return tag_str if len(tag_str) > 1 else None


def get_long_form(df, column_name, clean_func=None):
    """
    1. Parses the raw data (handles strings/lists/NaNs).
    2. Cleans and deduplicates items within each work.
    3. Explodes into a long-form DataFrame.
    """
    processed_rows = []

    # We only need these columns for the final analysis
    essential_cols = ['work_id', 'rating', 'full_read_date']

    for _, row in df.iterrows():
        # Step 1: Safe parse to get a real Python list
        raw_list = safe_parse_list(row.get(column_name, []))

        # Step 2: Clean and Deduplicate within this specific work
        cleaned_set = set()
        for item in raw_list:
            cleaned_item = clean_func(item) if clean_func else item
            if cleaned_item:
                cleaned_set.add(cleaned_item)

        # Step 3: Create a row for each unique item
        for item in cleaned_set:
            processed_rows.append({
                'work_id': row.get('work_id'),
                'full_read_date': row.get('full_read_date'),
                'rating': row.get('rating'),
                'attribute_value': item
            })

    return pd.DataFrame(processed_rows)




# =========================================================
# Helper: auto-tune max_words
# =========================================================
def choose_max_words(freq_dict, width=1600, height=800, coverage_target=0.85):
    counts = pd.Series(freq_dict).sort_values(ascending=False)

    coverage_limit = (
        (counts.cumsum() / counts.sum() <= coverage_target)
        .sum()
    )

    area_limit = int((width * height) / 20000)

    return max(15, min(coverage_limit, area_limit))


# =========================================================
# Helper: auto-tune margin
# =========================================================
def choose_margin(max_words, min_margin=4, max_margin=14):
    min_words, max_words_ref = 20, 120
    mw = np.clip(max_words, min_words, max_words_ref)

    margin = min_margin + (max_margin - min_margin) * (
        (mw - min_words) / (max_words_ref - min_words)
    )

    return int(round(margin))


# =========================================================
# Helper: CJK-safe font selection
# =========================================================
def find_best_font():
    fonts = (
        font_manager.findSystemFonts(fontext="ttf") +
        font_manager.findSystemFonts(fontext="ttc")
    )

    priority = [
        "notosanscjk",
        "notosans",
        "arialunicode",
        "dejavusans",
    ]

    fonts_lower = [f.lower() for f in fonts]

    # Prefer bold
    for key in priority:
        for i, name in enumerate(fonts_lower):
            if key in name and "bold" in name:
                return fonts[i]

    # Fallback to regular
    for key in priority:
        for i, name in enumerate(fonts_lower):
            if key in name:
                return fonts[i]

    return fonts[0] if fonts else None


# ---------- WordCloud Generation ----------
def generate_wc(freq_dict, colors="Reds", custom_mask=None, stop_words=None):
    """
    Generate a WordCloud object from a frequency dictionary with optional customization.

    This function creates a WordCloud using frequencies of terms provided in `freq_dict`.
    It supports stop word filtering, custom masks, color maps, and auto-tuned parameters
    for maximum words, margins, and font selection.

    Parameters
    ----------
    freq_dict : dict
        A dictionary where keys are words/phrases and values are their frequencies.
        Example: {'Harry': 50, 'Hermione': 40, 'Ron': 35}

    colors : str, optional (default="Reds")
        The Matplotlib colormap to use for the word colors.
        Can also accept a list of colors.

    custom_mask : array-like or None, optional (default=None)
        A 2D numpy array or image to shape the word cloud.
        Non-zero values indicate where words can be drawn.

    stop_words : set or list of str, optional (default=None)
        Words to exclude from the word cloud.
        Comparison is case-insensitive.

    Returns
    -------
    WordCloud
        A WordCloud object from the `wordcloud` library, ready for plotting or saving.

    Notes
    -----
    - The function uses helper functions `choose_max_words`, `choose_margin`, and `find_best_font`
      to automatically adjust layout parameters based on the frequency dictionary.
    - Collocations are disabled to avoid multi-word combinations.
    - Relative scaling is set to 0.3 to balance size differences between frequent and less frequent words.
    - Higher scale (3) is used for better rendering resolution.

    Example
    -------
    >>> freq_dict = {'Harry': 50, 'Hermione': 40, 'Ron': 35}
    >>> wc = generate_wc(freq_dict, colors="Blues", stop_words={"the", "and"})
    >>> plt.imshow(wc, interpolation="bilinear")
    >>> plt.axis("off")
    >>> plt.show()
    """
    # Filter stop words
    if stop_words:
        freq_dict = {k: v for k, v in freq_dict.items() if k.lower() not in stop_words}

    # Use auto-tuning helpers
    max_words = choose_max_words(freq_dict)
    margin = choose_margin(max_words)
    font_path = find_best_font()

    wc = WordCloud(
        background_color=AO3_BG_LIGHT,
        max_words=max_words,
        scale=3,  # Higher scale for better resolution
        margin=margin,
        width=1600,
        height=800,
        font_path=font_path,
        mask=custom_mask,
        colormap=colors,
        collocations=False,
        relative_scaling=0.3
    )

    wc.generate_from_frequencies(freq_dict)
    return wc



# --- 2. Reusable Data Processing Utilities ---
# 1. Prepare cleaners
cleaners = {
    'fandoms': clean_fandom,
    'relationships': clean_tag_generic,
    'characters': clean_tag_generic,
    'freeforms': clean_tag_generic
}

# 2. Generate the cleaned long-form dataframes
cleaned_data = {}
for col, func in cleaners.items():
    cleaned_data[col] = get_long_form(df, col, clean_func=func)

## Visualization 1: Summary Stats



In [ ]:
# @title AO3 Wrapped Cover Page

# --- 1. Quick Stats Calculation ---
total_works = len(df)
total_words = df['word_count'].sum()
df['word_count'] = pd.to_numeric(df['word_count'], errors='coerce')
longest_fic_words = int(df['word_count'].max())
df['last_updated_dt'] = pd.to_datetime(df['last_updated'], format='%d %b %Y', errors='coerce')
oldest_fic_year = df['last_updated_dt'].min().strftime('%b %Y')

# --- 2. Main Color Settings ---
AO3_BG_LIGHT = '#FFFFFF'
AO3_TEXT_DARK = '#000000'
AO3_ACCENT_RED = '#990000'
AO3_GRID_LIGHT = '#EEEEEE'
AO3_OTHER_GRAY = '#C0C0C0'
RATING_COLORS = ['#94c90e', '#ead70f', '#eb7400', '#a80403'] # GA, TEEN, MATURE, EXPLICIT

# --- 3. Canvas setup (1:1 ratio) ---
fig = plt.figure(figsize=(12, 12), facecolor=AO3_BG_LIGHT)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis("off")

# --- 4. Background Accents ---
# Top brand bar
ax.add_patch(plt.Rectangle((0, 0.94), 1, 0.1, color=AO3_ACCENT_RED, transform=ax.transAxes))

# Bottom rating-inspired strip
for i, color in enumerate(RATING_COLORS):
    ax.add_patch(plt.Rectangle((i*0.25, 0), 0.25, 0.05, color=color, transform=ax.transAxes))

# --- 5. Header (Using add_text_locked) ---
add_text_locked(fig, 0.08, 0.86, f"{YEAR_LIMIT}: A YEAR IN THE ARCHIVE",
                fontsize=48, fontweight="bold", color=AO3_TEXT_DARK, ha="left")

add_text_locked(fig, 0.08, 0.8, "PERSONAL READING JOURNEY",
                fontsize=32, color=AO3_TEXT_DARK, ha="left",
                alpha=0.6)

# --- 6. Core Stats (High Impact) ---
# Word Count
add_text_locked(fig, 0.5, 0.66, f"{total_words}",
                ha="center", fontsize=72, fontweight="bold", color=AO3_ACCENT_RED)
add_text_locked(fig, 0.5, 0.6, "WORDS VISITED",
                ha="center", fontsize=24, color=AO3_TEXT_DARK,
                alpha=0.6, fontweight="semibold")

# Works Count
add_text_locked(fig, 0.5, 0.49, f"{total_works}",
                ha="center", fontsize=72, fontweight="bold", color=AO3_ACCENT_RED)
add_text_locked(fig, 0.5, 0.43, "WORKS EXPLORED",
                ha="center", fontsize=24, color=AO3_TEXT_DARK,
                alpha=0.6, fontweight="semibold")

# Divider Line
fig.lines.append(plt.Line2D([0.15, 0.85], [0.46, 0.46], lw=1.5, color=AO3_OTHER_GRAY, alpha=0.5))

# --- 7. Flavor Stats (Structured Layout) ---
# Left Column: Longest Fic
add_text_locked(fig, 0.15, 0.315, "LONGEST JOURNEY", ha="left", fontsize=22,
                color=AO3_TEXT_DARK, alpha=0.6, fontweight="bold")
add_text_locked(fig, 0.15, 0.27, f"{longest_fic_words}", ha="left", fontsize=32, fontweight="bold", color=AO3_TEXT_DARK)
add_text_locked(fig, 0.15, 0.225, "WORDS IN A SINGLE WORK", ha="left", fontsize=20,
                color=AO3_TEXT_DARK, alpha=0.6)

# Right Column: Oldest Fic
add_text_locked(fig, 0.85, 0.315, "ARCHIVAL FIND", ha="right", fontsize=22,
                color=AO3_TEXT_DARK, alpha=0.6, fontweight="bold")
add_text_locked(fig, 0.85, 0.27, oldest_fic_year.upper(), ha="right", fontsize=32, fontweight="bold", color=AO3_TEXT_DARK)
add_text_locked(fig, 0.85, 0.225, "OLDEST WORK VISITED", ha="right", fontsize=20,
                color=AO3_TEXT_DARK, alpha=0.6)

# --- 8. Footer ---
add_text_locked(fig, 0.5, 0.09, "BASED ON WORKS YOU OPENED THIS YEAR",
                ha="center", fontsize=14, color=AO3_TEXT_DARK, alpha=0.5)

# The "Pro" touch
add_text_locked(fig, 0.95, 0.025, f"AO3 WRAPPED @ {YEAR_LIMIT}",
                ha="right", fontsize=12, color=AO3_BG_LIGHT, alpha=0.8)

# --- 9. Save

# --- Save & Show ---
plt.savefig("ao3_wrapped_cover.png", dpi=300, bbox_inches="tight", facecolor=AO3_BG_LIGHT)
plt.show()

## Visualization 2: Daily Work Opened Heatmap

In [ ]:
# @title Daily Work Opened Heatmap

daily_works = get_daily_data(df, value_col='work_id', agg_func='nunique')
daily_works_ts = reindex_time_series(daily_works)

# --- 1. Create the Pivot Table ---

# Rows: day of week (0=Mon to 6=Sun)
# Columns: week of year
# Values: the daily count of works finished ('daily_metric')
heatmap_data = daily_works_ts.pivot_table(
    index='dayofweek',
    columns='weekofyear',
    values='daily_metric',
    aggfunc='sum' # Use sum, though it should just be one value per cell
)

# Fill any NaN values (should be none if reindexing was done, but safe to keep)
heatmap_data = heatmap_data.fillna(0)


# --- 2. Reorder Rows for Standard Calendar View ---

# By default, Pandas dayofweek is Monday=0 to Sunday=6.
# For a typical calendar view (Monday at the top, Sunday at the bottom),
# we need to reverse the order of the rows.

# Note: heatmap_data.iloc[::-1] reverses the rows.
heatmap_data = heatmap_data.iloc[::-1]


# --- 3. Define Weekday Labels (Reordered) ---

# Define the labels to match the reversed order:
weekday_labels = ['Sun', 'Sat', 'Fri', 'Thu', 'Wed', 'Tue', 'Mon']

# 1. Prepare data for Mon-Sun order
# heatmap_data is currently Sun-Mon (due to iloc[::-1] in 4oO0RtpobUUf)
# Reverse it again to get Mon-Sun for display
heatmap_data_mon_sun = heatmap_data.iloc[::-1]
weekday_labels_mon_sun = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# --- 1. Figure Setup and Plotting ---
plt.figure(figsize=(20, 4)) # Wide and short for the calendar look

ax = sns.heatmap(
    heatmap_data_mon_sun, # Use the new data with Mon-Sun order
    cmap=AO3_HEATMAP_CMAP,
    linewidths=2,                   # Increased line width for cell borders
    linecolor=AO3_BG_LIGHT,         # White lines between cells
    square=True,                    # Ensure cells are square
    cbar=True,                      # CRITICAL: Add the color bar
    cbar_kws={
        'label': 'Works Opened',
        'shrink': 0.72,
        'pad': 0.015}, # Label for the color bar
    yticklabels=weekday_labels_mon_sun,     # Use the new labels for Mon-Sun
    xticklabels=False,              # Remove week numbers
    vmin=0                          # Ensure 0 starts at the lightest color
)

# Reduce colorbar label font size
cbar = ax.collections[0].colorbar
cbar.set_label('Works Finished', fontsize=10)
cbar.ax.tick_params(labelsize=9)

# --- 2. Axis and Tick Cleanup (Minimalism) ---

# Remove axis labels and tick marks
ax.set_xlabel('')
ax.set_ylabel('')
ax.tick_params(axis='x', length=0)
ax.tick_params(axis='y', length=0)

# Rotate Y-labels to be horizontal and align correctly
plt.yticks(rotation=0)

# --- 3. Adding Month Annotations ---

# The utility function needs the index of the original daily data (before pivot)
# to correctly calculate week numbers relative to the start of the year.
# Assuming 'daily_works_ts' is the DataFrame from reindex_time_series.
first_day_of_data = daily_works_ts.index.min()
last_day_of_data = daily_works_ts.index.max()

# Recalculate full date range needed for annotation logic
date_index_for_ann = pd.date_range(start=first_day_of_data, end=last_day_of_data, freq='D')

# Get month positions and labels
month_starts, month_labels = get_month_annotations(date_index_for_ann)

# Place the month labels above the weeks they start in
for pos, label in zip(month_starts, month_labels):
    ax.text(pos,
            len(weekday_labels_mon_sun) + 0.8,
            label, ha='left', va='bottom', fontsize=12, color=AO3_TEXT_DARK)


# --- 4. Final Title and Layout ---

# Title (aligned left, using locked font)
add_text_locked(
    obj=ax,
    x=0,             # x position is ignored for titles
    y=0,             # y position is ignored for titles
    text='THE DAILY PULSE: FICS OPENED PER DAY',
    fontsize=TITLE_FONT_SIZE,
    fontweight='bold',
    color=AO3_TEXT_DARK,
    ha='left',       # Title alignment: left, center, right
    is_title=True,
    pad=PADDING
)

# # Subtitle (positioned above the plot, axes coordinates)
ax.text(
    0, 1.08,  # Position: x=0 (left-aligned), y=1.08 (above the plot)
    'Your reading curiosity, traced across the calendar one day at a time.',
    transform=ax.transAxes,  # Use axes coordinates (0-1 range)
    fontsize=SUBTITLE_FONT_SIZE,
    color=AO3_TEXT_DARK,
    ha='left',  # Horizontal alignment
    va='bottom'  # Vertical alignment
)

# Ensure plot limits are clean and axis scaling is correct
# For Mon-Sun order, day 0 (Mon) should be at the top of the y-axis (index 0 in heatmap_data_mon_sun)
ax.set_ylim(len(weekday_labels_mon_sun), 0) # Corrected y-axis limit for Mon at top
ax.set_xlim(0, len(heatmap_data.columns)) # Week columns

plt.tight_layout()

# --- 5. Save High-Resolution Output ---
plt.savefig(
    'daily_works_heatmap.png',
    dpi=OUTPUT_DPI,
    bbox_inches='tight',
    facecolor='white'
)

plt.show()

## Visualization 3: Deleted Works

In [ ]:
# @title Monthly Deleted Work Bar Chart

# --- Handle empty deleted work DataFrame ---
if df_del.empty or df_del['last_visited_date'].dropna().empty:
    # Create an "empty" plotting series for consistency (0 bars for 12 months)
    import datetime
    today = pd.Timestamp.today()
    months = pd.period_range(today - pd.DateOffset(months=11), today, freq='M')
    plotting_series = pd.Series(0, index=months.to_timestamp())

    # Base gray bars for empty plot
    BASE_GRAY = '#C0C0C0'
    bar_colors = [BASE_GRAY] * len(plotting_series)

    # Create the figure
    plt.figure(figsize=(14, 6), facecolor=AO3_BG_LIGHT)
    ax = plt.gca()
    ax.bar(plotting_series.index, plotting_series.values, color=bar_colors, width=19)

    # Title
    TITLE_TEXT = 'GHOST READING HISTORY: DELETED WORKS PER MONTH'
    add_text_locked(
        obj=ax,
        x=0,
        y=0,
        text=TITLE_TEXT,
        fontsize=TITLE_FONT_SIZE,
        fontweight='bold',
        color=AO3_TEXT_DARK,
        ha='left',
        is_title=True,
        pad=PADDING
    )

    # Fun / interesting subtitle
    ax.text(
        0, 1.06,
        "Good news! You haven't lost any works yet — your archive remains pristine.",
        transform=ax.transAxes,
        ha='left',
        va='bottom',
        fontsize=SUBTITLE_FONT_SIZE,
        color=AO3_TEXT_DARK,
        wrap=True
    )

    # Labels & formatting
    ax.set_xlabel('Month Read', fontsize=14, color=AO3_TEXT_DARK)
    ax.set_ylabel('Number of Deleted Works', fontsize=14, color=AO3_TEXT_DARK)
    ax.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%b'))
    sns.despine(ax=ax, top=True, right=True)
    ax.grid(axis='y', linestyle='--', alpha=0.5, color=AO3_GRID_LIGHT)
    ax.set_ylim(0, 5)  # Small fixed y-range to show minimal bars
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    # Save high-resolution PNG
    plt.savefig('deleted_works_per_month.png', dpi=OUTPUT_DPI, bbox_inches='tight', facecolor=AO3_BG_LIGHT)
    plt.show()

else:
    # --- Existing plotting code for non-empty df_del ---
    df_del['last_visited_date'] = pd.to_datetime(df_del['last_visited_date'])
    df_del['month'] = df_del['last_visited_date'].dt.to_period('M')
    monthly_deleted_counts = df_del.groupby('month').size()
    monthly_deleted_counts.name = 'deleted_work_count'

    full_month_range = pd.period_range(
        start=monthly_deleted_counts.index.min(),
        end=monthly_deleted_counts.index.max(),
        freq='M'
    )
    monthly_deleted_counts = monthly_deleted_counts.reindex(full_month_range, fill_value=0)
    plotting_series = monthly_deleted_counts.to_timestamp()

    max_deleted_count = plotting_series.max()
    max_month_index = plotting_series.idxmax()
    max_deleted_month = max_month_index.strftime('%b %Y')

    BASE_GRAY = '#C0C0C0'
    bar_colors = [
        AO3_ACCENT_RED if date_index == max_month_index else BASE_GRAY
        for date_index in plotting_series.index
    ]

    plt.figure(figsize=(14, 6), facecolor=AO3_BG_LIGHT)
    ax = plt.gca()
    ax.bar(plotting_series.index, plotting_series.values, color=bar_colors, width=19)

    TITLE_TEXT = 'GHOST READING HISTORY: DELETED WORKS PER MONTH'
    add_text_locked(
        obj=ax,
        x=0,
        y=0,
        text=TITLE_TEXT,
        fontsize=TITLE_FONT_SIZE,
        fontweight='bold',
        color=AO3_TEXT_DARK,
        ha='left',
        is_title=True,
        pad=PADDING
    )

    subtitle_text = (
        f"The most tragic loss occurred in {max_deleted_month}, "
        f"where {max_deleted_count} works vanished from the archive."
    )
    ax.text(0, 1.06, subtitle_text, transform=ax.transAxes, ha='left', va='bottom', fontsize=SUBTITLE_FONT_SIZE, color=AO3_TEXT_DARK, wrap=True)

    # ax.set_xlabel('Month Read', fontsize=14, color=AO3_TEXT_DARK)
    ax.set_xlabel("")
    ax.set_ylabel('Number of Deleted Works', fontsize=14, color=AO3_TEXT_DARK)
    ax.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%b'))
    sns.despine(ax=ax, top=True, right=True)
    ax.grid(axis='y', linestyle='--', alpha=0.5, color=AO3_GRID_LIGHT)
    ax.set_ylim(bottom=0)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig('deleted_works_per_month.png', dpi=OUTPUT_DPI, bbox_inches='tight', facecolor=AO3_BG_LIGHT)
    plt.show()

## Visualization 4: Top Fandoms Stream Chart

In [ ]:
# @title Evolution of Interests: Top Fandoms over Time

# Define OUTPUT_DPI if not already in your config
# OUTPUT_DPI = 300

NUM_TOP_FANDOMS = 8

def plot_proportional_stream_chart(
    long_df,
    category_label='Fandom',
    num_top=NUM_TOP_FANDOMS,
    save_path="fandom_stream_chart.png"
):
    df_temp = long_df.copy()

    # --- 1. Preparation ---
    df_temp['full_read_date'] = pd.to_datetime(df_temp['full_read_date'])
    df_temp['month'] = df_temp['full_read_date'].dt.to_period('M')

    top_categories = df_temp['attribute_value'].value_counts().head(num_top).index.tolist()
    df_filtered = df_temp[df_temp['attribute_value'].isin(top_categories)].copy()

    monthly_counts = df_filtered.groupby(['month', 'attribute_value']).size().unstack(fill_value=0)
    monthly_counts = monthly_counts[top_categories]
    pivot_norm = monthly_counts.div(monthly_counts.sum(axis=1), axis=0).fillna(0)

    # --- 2. Narrative Logic ---
    start_val = pivot_norm.iloc[0].idxmax()
    end_val = pivot_norm.iloc[-1].idxmax()

    mapping = {"Original Work": "Published Fiction"}
    start_disp = mapping.get(start_val, start_val)
    end_disp = mapping.get(end_val, end_val)

    TITLE_TEXT = f"EVOLUTION OF INTERESTS: TOP {category_label.upper()}S IN {YEAR_LIMIT}"
    if start_val == end_val:
        SUBTITLE_TEXT = f"A portrait of loyalty: {start_disp} remained your defining interest throughout this period."
    else:
        SUBTITLE_TEXT = f"Tracing the shift from your {start_disp} era to your current {end_disp} fixation."

    # --- 3. Plotting ---
    # We increase the figure size slightly for better resolution handling
    fig, ax = plt.subplots(figsize=(18, 9), facecolor='#FFFFFF')

    colors = sns.color_palette("Set2", n_colors=len(top_categories))
    pivot_norm.plot(kind='area', stacked=True, ax=ax, color=colors, alpha=0.7, linewidth=0)

    # Titles and Subtitles
    add_text_locked(
        obj=ax,
        x=0,
        y=0,
        text=TITLE_TEXT,
        fontsize=TITLE_FONT_SIZE,
        fontweight='bold',
        color=AO3_TEXT_DARK,
        ha='left',
        is_title=True,
        pad=PADDING
    )
    ax.text(x=0, y=1.03, s=SUBTITLE_TEXT, transform=ax.transAxes,
            fontsize=SUBTITLE_FONT_SIZE, fontweight='medium', color='#333333', ha='left', va='bottom')

    # Formatting
    # Ensure the axis has room to breathe without clipping labels
    ax.margins(x=0.02) # Small 2% margin so the first/last labels have space

    x_values = pivot_norm.index.to_timestamp()
    month_labels = [m.strftime('%b') for m in pivot_norm.index]

    ax.set_xticks(x_values)
    ax.set_xticklabels(month_labels, rotation=0, fontsize=12)
    ax.set_xlabel("")

    # Y-Axis: Percentages and Label
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, pos: f'{int(x*100)}%'))
    ax.set_ylabel(f"Share of Top {num_top} {category_label}s", fontsize=12)
    ax.set_ylim(0, 1)

    ax.legend(title=category_label, bbox_to_anchor=(1.02, 1), loc='upper left', frameon=False)
    sns.despine(ax=ax, left=True, bottom=False)
    ax.grid(axis='x', linestyle='--', alpha=0.3)

    # --- 4. Save and Show ---
    plt.tight_layout()

    # # Save the figure BEFORE plt.show()
    plt.savefig(
        save_path,
        dpi=OUTPUT_DPI,
        bbox_inches='tight',
        facecolor=fig.get_facecolor(),
        transparent=False
    )
    # print(f"Successfully saved chart to: {save_path} at {OUTPUT_DPI} DPI")

    plt.show()

# --- Execution ---
plot_proportional_stream_chart(
    long_df=cleaned_data['fandoms'],
    category_label='Fandom',
    num_top=NUM_TOP_FANDOMS
)

## Visualization 5: Favourite Fics and Authors

In [ ]:
# @title Favourite Fics and Authors

# ---------- Data Preparation ----------

# 1. Top 5 Visited Works
df_top_works = df.sort_values(by='visited_count', ascending=False).head(5).copy()

# 2. Top 5 Authors (Excluding Aggregators)
exclude_authors = ['anonymous', 'orphan_account', 'Anonymous']
df_filtered_authors = df[~df['author'].isin(exclude_authors)].copy()
df_top_authors = df_filtered_authors.groupby('author')['work_id'].nunique().sort_values(ascending=False).head(5).reset_index()
df_top_authors.columns = ['author', 'unique_works_count']

# Stats for the technical note
anon_count = df[df['author'].str.lower() == 'anonymous']['work_id'].nunique()
orphan_count = df[df['author'] == 'orphan_account']['work_id'].nunique()

# ---------- Plotting ----------
# Define two specific shades of AO3 Red
RED_DEEP = '#990000'   # Classic AO3 Red
RED_VIVID = '#D00000'  # Slightly brighter Red

fig, axes = plt.subplots(1, 2, figsize=(18, 9), facecolor=AO3_BG_LIGHT)

# --- Subplot 1: Top 5 Visited Works ---
ax1 = axes[0]
sns.barplot(data=df_top_works, x='visited_count', y='title', ax=ax1, color=RED_DEEP)
add_text_locked(
    obj=ax1,
    x=0,
    y=0,
    text='MOST VISITED WORKS',
    fontsize=14,
    fontweight='bold',
    color=AO3_TEXT_DARK,
    ha='left',
    is_title=True,
    pad=10
)
ax1.set_xlabel('Total Visits', fontsize=10, color=AO3_TEXT_DARK)
ax1.set_ylabel('') # Clean Y-axis

# --- Subplot 2: Top 5 Authors ---
ax2 = axes[1]
sns.barplot(data=df_top_authors, x='unique_works_count', y='author', ax=ax2, color=RED_VIVID)
add_text_locked(
    obj=ax2,
    x=0,
    y=0,
    text='MOST VISITED AUTHORS',
    fontsize=14,
    fontweight='bold',
    color=AO3_TEXT_DARK,
    ha='left',
    is_title=True,
    pad=10
)
ax2.set_xlabel('Unique Works Opened', fontsize=10, color=AO3_TEXT_DARK)
ax2.set_ylabel('')

# --- Global Styling & Annotations ---
for ax in axes:
    sns.despine(ax=ax, left=True)
    ax.tick_params(axis='both', colors=AO3_TEXT_DARK)

    # Place count number INSIDE the bar in white
    for container in ax.containers:
        ax.bar_label(container, label_type='center', color='white',
                     fontweight='bold', fontsize=12)

# --- Integrated Titles & Narratives (Matching WordCloud Style) ---
# General Title
add_text_locked(
    obj=fig,
    x=0.08, y=0.96,
    text='FAVOURITE FICS: TOP WORKS & AUTHORS',
    fontsize=TITLE_FONT_SIZE, fontweight='bold', color=AO3_TEXT_DARK, ha='left'
)

# Narrative Subtitle
top_work = df_top_works.iloc[0]['title']
top_auth = df_top_authors.iloc[0]['author']
subtitle_text = (
    "If only you could leave multiple kudos on a single fic! Maybe leave a comment to let the authors know you appreciate their work?"
)
fig.text(
    x=0.08, y=0.91,
    s=subtitle_text,
    fontsize=SUBTITLE_FONT_SIZE, color='#000000', ha='left'
)

# --- Build dynamic note text ---
parts = []

if anon_count > 0:
    parts.append(f"Anonymous accounts ({anon_count} work{'s' if anon_count != 1 else ''})")

if orphan_count > 0:
    parts.append(f"Orphan accounts ({orphan_count} work{'s' if orphan_count != 1 else ''})")

if parts:  # Only show note if there is at least one category
    if len(parts) == 1:
        note_intro = parts[0] + " doesn’t appear in the top author list."
    else:
        note_intro = " and ".join(parts) + " don’t appear in the top author list."

    note_text = note_intro + " But we still thank the anons for their brilliant works."

    fig.text(
        0.5, 0.02,
        note_text,
        ha='center',
        fontsize=14,
        color=AO3_TEXT_DARK,
        style='italic',
        alpha=0.6
    )
# If both counts are 0, nothing is displayed


plt.tight_layout(rect=[0.05, 0.05, 0.95, 0.90]) # Padding for titles
plt.savefig("top_works_authors.png", dpi=OUTPUT_DPI, bbox_inches='tight')
plt.show()

## Visualization 6: Top Fandoms

In [ ]:
# @title Top Fandoms Word Cloud
from matplotlib import font_manager, colormaps

# ---------- Process Data using the Long-Form Utility ----------
# Generate frequency dictionary from the cleaned long-form data
fandom_counts = cleaned_data['fandoms']['attribute_value'].value_counts().to_dict()

# Create a new colormap using only the 25% to 100% range of RdPu
colors = plt.get_cmap('Blues')(np.linspace(0.25, 1.0, 256))
new_cmap = mcolors.ListedColormap(colors)

# Generate
fandom_wordcloud = generate_wc(fandom_counts, colors=new_cmap)

# ---------- Plotting ----------
plt.figure(figsize=(16, 8), facecolor=AO3_BG_LIGHT)
ax = plt.gca()

ax.imshow(fandom_wordcloud, interpolation='bilinear')
ax.axis("off")

# Title
add_text_locked(
    obj=ax,
    x=0,            # ignored for titles
    y=0,            # ignored for titles
    text='FANDOM UNIVERSE: YOUR READING LANDSCAPE',
    fontsize=TITLE_FONT_SIZE,
    fontweight='bold',
    color=AO3_TEXT_DARK,
    ha='left',      # maps to ax.set_title(loc='left')
    is_title=True,
    pad=PADDING
)

# Narrative Subtitle
top_fandoms = sorted(fandom_counts.items(), key=lambda x: x[1], reverse=True)[:5]
top_fandoms_str = ', '.join([f[0] for f in top_fandoms])

# Count total works
total_works = len(df)

ax.text(
    x=0, y=1.04,
    s=f"Across {total_works} works, your heart belongs to: {top_fandoms_str}.",
    transform=ax.transAxes, ha='left', va='bottom', fontsize=SUBTITLE_FONT_SIZE,
    color='#000000'
)

# Save high-res
plt.savefig("fandom_wordcloud.png", dpi=OUTPUT_DPI, bbox_inches='tight')
plt.show()

## Visualization 7: Top Characters

In [ ]:
# @title Top Characters Word Cloud
from matplotlib import font_manager, colormaps

# ---------- Process Data using the Long-Form Utility ----------
# Generate frequency dictionary from the cleaned long-form data
char_counts = cleaned_data['characters']['attribute_value'].value_counts().to_dict()

# Create a new colormap using only the 25% to 100% range of RdPu
colors = plt.get_cmap('Purples')(np.linspace(0.25, 1.0, 256))
new_cmap = mcolors.ListedColormap(colors)


# Generate
character_wordcloud = generate_wc(char_counts, colors=new_cmap)

# ---------- Plotting ----------
plt.figure(figsize=(16, 8), facecolor=AO3_BG_LIGHT)
ax = plt.gca()

ax.imshow(character_wordcloud, interpolation='bilinear')
ax.axis("off")

# Title
add_text_locked(
    obj=ax,
    x=0,            # ignored for titles
    y=0,            # ignored for titles
    text='CHARACTER POWER RANKINGS: YOUR MOST READ',
    fontsize=TITLE_FONT_SIZE,
    fontweight='bold',
    color=AO3_TEXT_DARK,
    ha='left',      # maps to ax.set_title(loc='left')
    is_title=True,
    pad=PADDING
)

# Narrative Subtitle
top_5 = list(character_wordcloud.words_.keys())[:5]
top_str = ', '.join(top_5)
ax.text(
    x=0, y=1.04, s=f"Your reading focus revolves heavily around: {top_str}.",
    transform=ax.transAxes, ha='left', va='bottom', fontsize=SUBTITLE_FONT_SIZE,
    color='#000000'
)

# Save high-res
plt.savefig("character_wordcloud.png", dpi=OUTPUT_DPI, bbox_inches='tight')
plt.show()

## Visualization 8: Top Freeform Tags (By Rating)

In [ ]:
# @title Top Freeforms Word Cloud
from matplotlib import font_manager, colormaps

# --- Configuration ---
# AO3 Yellow: #ead70f
# Create a custom gradient for Yellow to ensure it doesn't vanish on white
yellow_colors = ["#FFF394", "#FFE761", "#DAA520"] # Light -> Icon Color -> Dark
yellow_cmap = mcolors.LinearSegmentedColormap.from_list("AO3_Yellow", yellow_colors)

RATINGS = [
    'General Audiences',
    'Teen And Up Audiences',
    'Mature',
    'Explicit',
    'Not Rated'
    ]

RATING_CMAPS = {
    'Explicit': 'Reds',
    'Mature': 'Oranges',
    'Teen And Up Audiences': yellow_cmap,
    'General Audiences': 'Greens',
    'Not Rated': 'Greys'
}

# --- Execution ---
tags_df = cleaned_data['freeforms'].copy()

for rating in RATINGS:
    # 1. Get frequency dict for this specific rating
    # We filter the long-form tags by the rating column
    rating_counts = tags_df[tags_df['rating'] == rating]['attribute_value'].value_counts().to_dict()

    if not rating_counts:
        print(f"Skipping {rating}: No tags found.")
        continue

    # 2. Create the cropped colormap (0.25 to 1.0 to ensure readability)
    base_cmap_name = RATING_CMAPS.get(rating, 'Blues')
    colors = plt.get_cmap(base_cmap_name)(np.linspace(0.25, 1.0, 256))
    rating_cmap = mcolors.ListedColormap(colors)

    # 3. Generate Word Cloud using your existing helper
    # Assuming your generate_wc takes (freq_dict, colormap=...)
    wc = generate_wc(rating_counts, colors=rating_cmap)

    # 4. Plotting
    plt.figure(figsize=(16, 8), facecolor=AO3_BG_LIGHT)
    ax = plt.gca()

    ax.imshow(wc, interpolation='bilinear')
    ax.axis("off")

    # Dynamic Title
    add_text_locked(
        obj=ax,
        x=0,            # ignored for titles
        y=0,            # ignored for titles
        text=f'TAG LANDSCAPE: {rating.upper()}',
        fontsize=TITLE_FONT_SIZE,
        fontweight='bold',
        color=AO3_TEXT_DARK,
        ha='left',      # maps to ax.set_title(loc='left')
        is_title=True,
        pad=PADDING
    )



    # Subtitle with top 5 tags for this rating
    top_tags = list(wc.words_.keys())[:5]
    top_tags_str = ', '.join(top_tags)

    ax.text(
        x=0, y=1.04,
        s=f"Common themes in your {rating} reads: {top_tags_str}.",
        transform=ax.transAxes, ha='left', va='bottom',
        fontsize=SUBTITLE_FONT_SIZE, color='#333333'
    )

    # 5. Save and Show
    file_name = f"wordcloud_rating_{rating.replace(' ', '_').lower()}.png"
    plt.savefig(file_name, dpi=OUTPUT_DPI, bbox_inches='tight', facecolor=AO3_BG_LIGHT)

    # print(f"Generated Word Cloud for: {rating}")
    plt.show()

# 🎉 That’s the end of your AO3 Wrapped!

Thanks for taking a tour through your reading history.  
Your AO3 Wrapped captures the stories, fandoms, and moments that shaped your reading journey, highlighting patterns you might not have noticed before.

Whether you stop here or keep exploring, this Wrapped is yours to enjoy.

**Everything below is optional** — consider it bonus content if you’re in the mood to explore further.


&nbsp;  
&nbsp;  

---

## ⚙️ Want to Customize? *(Optional)*

You can tweak how this Wrapped is generated — for example:

- 📅 Change the analysis year (`YEAR_LIMIT`)
- 🏷️ Adjust how many fandoms are shown (`NUM_TOP_FANDOMS`)
- ⏱️ Control scraping limits and delays

### How to customize
1. **Scroll back up** to the **SETUP** cell  
2. Click **“Show code”** to expand it  
3. Find the section labeled **`CONFIGURATION SETTINGS`**  
4. Edit the values you want  
5. Run the cell again to apply your changes

&nbsp;  
&nbsp;  

---

## 🔍 Still Want to Dig Deeper? *(Optional)*

The following cells let you **filter and zoom in** on specific parts of your reading history, such as:

- A single fandom
- A particular language
- Specific characters or relationships
- Custom time ranges

You’ll find a few examples showing how to use the filter functions and how to generate visualizations from the filtered DataFrames.  
These steps are completely optional — feel free to skip them if you’re happy with the overview above :)

&nbsp;  
&nbsp;  
&nbsp;  


In [ ]:
# @title Filter Function
def filter_data(
    df,
    ratings_to_include=None,
    fandoms_to_include=None,
    min_visited_count=0,
    min_word_count=0,
    languages_to_include=None # New parameter for language filter
):
    """
    Applies multiple filters (Rating, Fandom, Visited Count, Word Count, Language)
    to the AO3 data in a single pass.
    """
    current_df = df.copy() # Work on a copy to avoid side effects

    # 1. Apply Rating Filter
    if ratings_to_include:
        current_df = current_df[current_df['rating'].isin(ratings_to_include)]

    # 2. Apply Fandom Filter
    if fandoms_to_include:
        def has_fandom(fandom_list):
            # Safely evaluate string representation of list if needed, then check
            if isinstance(fandom_list, str):
                try:
                    fandoms_parsed = ast.literal_eval(fandom_list)
                except (ValueError, SyntaxError):
                    fandoms_parsed = []
            else:
                fandoms_parsed = fandom_list

            return any(fandom in fandoms_to_include for fandom in fandoms_parsed)

        current_df = current_df[current_df['fandoms'].apply(has_fandom)]

    # 3. Apply Language Filter
    if languages_to_include:
        current_df = current_df[current_df['language'].isin(languages_to_include)]

    # 4. Apply Threshold Filters
    current_df = current_df[current_df['visited_count'] >= min_visited_count]
    current_df = current_df[current_df['word_count'] >= min_word_count]

    # Reindex current_df before returning
    current_df = current_df.reset_index(drop=True)

    return current_df

In [ ]:
# @title Examples to use the Filter Function
"""
Run any of these examples, or create your own custom filters!
"""

# Example 1: Only General Audiences works (SFW)
df_ga = filter_data(
    df,
    ratings_to_include=['General Audiences']
)

# Example 2: Works you re-read (visited 2+ times)
df_rereads = filter_data(
    df,
    min_visited_count=2
)

# Example 3: Long fics only (50k+ words)
df_longfic = filter_data(
    df,
    min_word_count=50000
)

# Example 4: English works only
df_english = filter_data(
    df,
    languages_to_include=['English']
)

# Example 5: Combined filters - Teen+ English longfics
df_custom = filter_data(
    df,
    ratings_to_include=['Teen And Up Audiences', 'General Audiences'],
    languages_to_include=['English'],
    min_word_count=20000
)

# 💡 TIP: Use filtered DataFrames in visualizations!
# For example, to see a wordcloud of only SFW characters:
# character_wordcloud = generate_cleaned_wordcloud(df_ga, column_name='characters')

In [ ]:
# @title Use Filtered Dataframes Examples - Character Stream Chart for English Works

# --- Example: Character Stream Chart for English Works (df_english) ---

# 1. Get work_ids from df_english
english_work_ids = df_english['work_id'].unique()

# 2. Filter cleaned_data['characters'] by these work_ids
filtered_characters_english_df = cleaned_data['characters'][cleaned_data['characters']['work_id'].isin(english_work_ids)].copy()

# 3. Plot the proportional stream chart for characters
if not filtered_characters_english_df.empty:
    plot_proportional_stream_chart(
        long_df=filtered_characters_english_df,
        category_label='Character',
        num_top=NUM_TOP_FANDOMS, # Reusing NUM_TOP_FANDOMS for characters for now
        save_path="characters_stream_chart_english.png"
    )
else:
    print("No character data found for English works to generate a stream chart.")

In [ ]:
# @title Use Filtered Dataframes Example - Relationship Word Cloud for Reread Works

# --- Example: Relationship Word Cloud for Reread Works (df_rereads) ---

# 1. Get work_ids from df_rereads
reread_work_ids = df_rereads['work_id'].unique()

# 2. Filter cleaned_data['relationships'] by these work_ids
filtered_relationships_df = cleaned_data['relationships'][cleaned_data['relationships']['work_id'].isin(reread_work_ids)].copy()

# 3. Generate frequency dictionary
relationship_counts_rereads = filtered_relationships_df['attribute_value'].value_counts().to_dict()

if not relationship_counts_rereads:
    print("No relationship tags found for re-read works to generate a word cloud.")
else:
    # 4. Create a new colormap (e.g., Reds, similar to other character/fandom WCs)
    colors = plt.get_cmap('Reds')(np.linspace(0.25, 1.0, 256))
    new_cmap = mcolors.ListedColormap(colors)

    # 5. Generate Word Cloud
    relationship_wordcloud_rereads = generate_wc(relationship_counts_rereads, colors=new_cmap)

    # 6. Plotting
    plt.figure(figsize=(16, 8), facecolor=AO3_BG_LIGHT)
    ax = plt.gca()

    ax.imshow(relationship_wordcloud_rereads, interpolation='bilinear')
    ax.axis("off")

    # Title
    add_text_locked(
        obj=ax,
        x=0,
        y=0,
        text='REREAD WORKS: RELATIONSHIP LANDSCAPE',
        fontsize=TITLE_FONT_SIZE,
        fontweight='bold',
        color=AO3_TEXT_DARK,
        ha='left',
        is_title=True,
        pad=PADDING
    )

    # Narrative Subtitle
    top_5 = list(relationship_wordcloud_rereads.words_.keys())[:5]
    top_str = ', '.join(top_5)
    ax.text(
        x=0, y=1.04, s=f"In your re-reads, your character pairings of choice are: {top_str}.",
        transform=ax.transAxes, ha='left', va='bottom', fontsize=SUBTITLE_FONT_SIZE,
        color='#000000'
    )

    # Save high-res
    # plt.savefig("relationship_wordcloud_rereads.png", dpi=OUTPUT_DPI, bbox_inches='tight')
    plt.show()

&nbsp;  
&nbsp;  


## GitHub Repo



[![View on GitHub](https://img.shields.io/badge/GitHub-Source-blue?logo=github)](https://github.com/anulomufa/AO3-Wrapped)

Feel free to watch, star, or fork this repository. You can also raise issues or contribute improvements via Pull Requests.
